In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = keras.utils.image_dataset_from_directory(
    directory='dataset/Training',
    labels='inferred',
    label_mode='categorical',
    batch_size = 10,
    image_size=(256,256)
)
test_data = keras.utils.image_dataset_from_directory(
    directory='dataset/Testing',
    labels='inferred',
    label_mode='categorical',
    batch_size = 10,
    image_size=(256,256)
)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [3]:
train_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [4]:
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout
from keras import Sequential

In [5]:
model = Sequential()

In [6]:
model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu',input_shape=(256,256,3)))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(train_data,epochs=5,validation_data=test_data)

Epoch 1/5
287/287 [==============================] - 40s 86ms/step - loss: 2.2620 - accuracy: 0.3829 - val_loss: 1.9092 - val_accuracy: 0.1853
Epoch 2/5
287/287 [==============================] - 23s 81ms/step - loss: 1.0444 - accuracy: 0.5132 - val_loss: 1.5161 - val_accuracy: 0.3579
Epoch 3/5
287/287 [==============================] - 23s 81ms/step - loss: 0.9151 - accuracy: 0.5641 - val_loss: 1.4514 - val_accuracy: 0.3807
Epoch 4/5
287/287 [==============================] - 23s 80ms/step - loss: 0.8337 - accuracy: 0.6000 - val_loss: 2.6149 - val_accuracy: 0.3122
Epoch 5/5
287/287 [==============================] - 23s 80ms/step - loss: 0.7387 - accuracy: 0.6540 - val_loss: 1.8068 - val_accuracy: 0.4569


In [5]:
import tensorflow as tf

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]